In [4]:
#Used for data that is already split into trigger1, trigger2 and label
def convert_to_rebel(data):
    data = data[data.label != str(0)]
    data = data.rename(columns={"sentence": "context"})
    data['triplets'] = '<triplet> ' + data['trigger1'] + ' <subj> ' + data['trigger2'] + ' <obj> ' + data['label'] #Add the suitable format
    data = data.drop(['trigger1', 'trigger2', 'label'], axis=1)
    return data

In [9]:
#Creating testset, using the non-synthetic sets, and checking if they occur in the train set of rebel
import pandas as pd
from sklearn.model_selection import train_test_split

enables = pd.read_csv('Data/rebel_v2/data/enables_stud.csv').drop(['verb'], axis=1)
enables = enables.rename(columns={"sentece": "sentence", "trigger_1": "trigger1", "trigger_2": "trigger2"})
intends = pd.read_csv('Data/rebel_v2/data/intend_left.csv')
intends['sentence'] = intends['sentence'].apply(lambda x: x[1:] if x.startswith(',') else x)
intends['sentence'] = intends['sentence'].apply(lambda x: x[2:] if x.startswith(' ,') else x)
prevents = pd.read_csv('Data/rebel_v2/data/prevents_stud.csv').drop(['verb'], axis=1)
prevents = prevents.rename(columns={"sentece": "sentence", "trigger_1": "trigger1", "trigger_2": "trigger2"})
causes = pd.read_csv('Data/rebel_v2/data/our_data.csv') #This also contains more than just cause
causes = causes.rename(columns={"sentece": "sentence", "trigger_1": "trigger1", "trigger_2": "trigger2"})

total = pd.concat([enables, intends, prevents, causes]).reset_index()
total = total[total['label'] != '0']
total = total.drop(['index', 'Unnamed: 0.1', 'Unnamed: 0', 'tag'], axis=1)
print(f"Total non-synthetic: {len(total)}")

#total.to_csv('Data/rebel_v2/data/rebel_merged_non_synthetic.csv')

rest_data, test_data = train_test_split(total, test_size=0.15, random_state=1, shuffle=True)
#test_data.to_csv('Data/rebel_v2/data/new_split/test.csv')


Total non-synthetic: 632


In [4]:
total['label'].value_counts()

cause      283
intend     137
enable     118
prevent     94
Name: label, dtype: int64

In [5]:
enables = pd.read_csv('Data/rebel_v2/data/enables.csv')
intends = pd.read_csv('Data/rebel_v2/data/intends.csv')
prevents = pd.read_csv('Data/rebel_v2/data/prevents.csv')
print(f"Instances - Enable: {len(enables)} - Prevent: {len(prevents)} - Intend: {len(intends)}")
#Cause is the same dataset as loaded above
#causes = causes.rename(columns={"sentece": "sentence", "trigger_1": "trigger1", "trigger_2": "trigger2"})
print(f"Total synthetic: {len(pd.concat([enables, intends, prevents]).reset_index(drop=True))}")

total = pd.concat([rest_data, enables, intends, prevents]).reset_index(drop=True)
print(f"Total before drop: {len(total)}")
total = total.drop(['Unnamed: 0.1', 'Unnamed: 0', 'tag'], axis= 1).drop_duplicates()
total = total[total['label'] != '0']
print(f"Total after drop: {len(total)}")

total.to_csv('Data/rebel_v2/data/new_split/train_val_combined.csv')
train_data, val_data = train_test_split(total, test_size=0.1, shuffle=True)
train_data.to_csv('Data/rebel_v2/data/new_split/train.csv')
val_data.to_csv('Data/rebel_v2/data/new_split/val.csv')


Instances - Enable: 455 - Prevent: 553 - Intend: 478
Total synthetic: 1486
Total before drop: 2023
Total after drop: 2001


In [10]:
train_data = pd.read_csv('Data/rebel_v2/data/new_split/train.csv').drop(['Unnamed: 0'], axis=1)
val_data = pd.read_csv('Data/rebel_v2/data/new_split/val.csv').drop(['Unnamed: 0'], axis=1)
test_data = pd.read_csv('Data/rebel_v2/data/new_split/test.csv').drop(['Unnamed: 0'], axis=1)

train_data = convert_to_rebel(train_data)
val_data = convert_to_rebel(val_data)
test_data = convert_to_rebel(test_data)

train_data.to_csv('Data/rebel_v2/data/new_split/train_rebel.csv')
val_data.to_csv('Data/rebel_v2/data/new_split/val_rebel.csv')
test_data.to_csv('Data/rebel_v2/data/new_split/test_rebel.csv')

In [12]:
import pandas as pd
data=pd.read_csv('Data/rebel_v2/data/new_split/train_val_combined.csv')
data['triplets'] = '<triplet> ' + data['trigger1'] + ' <subj> ' + data['trigger2'] + ' <obj> ' + data['label'] #Add the suitable format
data = data.sample(frac=1).reset_index(drop=True)
data.to_csv('Data/rebel_v2/data/new_split/train_val_combined_rebel.csv')